In [1]:
import numpy as np
import pandas as pd
import os
import glob
from tqdm import tqdm

In [2]:
from pathlib import Path

# Get the current working directory
current_path = Path.cwd()

# Get the parent directory
parent_path = current_path
data_paths = parent_path / 'datas'  

print(data_paths)
#
train_path = data_paths / 'train.csv'
test_path = data_paths / 'test.csv'

train_path_ts = data_paths / 'series_train.parquet'
test_path_ts = data_paths / 'series_test.parquet'

/home/koko/Bureau/projets/kaggle/cmi_probleamtic_use/CMI_problematic_internet_use/datas


In [ ]:
# Load datasets

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

## Pre processing

In [ ]:
features_to_keep = ['id', 'Basic_Demos-Age', 'Basic_Demos-Sex','CGAS-CGAS_Score', 'Physical-BMI', 'Physical-Diastolic_BP','Physical-HeartRate','Physical-Systolic_BP','Fitness_Endurance-Max_Stage','Fitness_Endurance-Time_Sec', 
                    'FGC-FGC_CU_Zone','FGC-FGC_GSND_Zone','FGC-FGC_GSD_Zone', 'FGC-FGC_PU_Zone', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR_Zone','FGC-FGC_TL_Zone',
                    'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI', 'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
                    'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num','BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM', 'BIA-BIA_TBW',
                    'PCIAT-PCIAT_03', 'PCIAT-PCIAT_04', 'PCIAT-PCIAT_05', 'PCIAT-PCIAT_06', 'PCIAT-PCIAT_07', 'PCIAT-PCIAT_08', 'PCIAT-PCIAT_09', 'PCIAT-PCIAT_10',
                    'PCIAT-PCIAT_11', 'PCIAT-PCIAT_12', 'PCIAT-PCIAT_13', 'PCIAT-PCIAT_14','PCIAT-PCIAT_15', 'PCIAT-PCIAT_16', 'PCIAT-PCIAT_17', 'PCIAT-PCIAT_18','PCIAT-PCIAT_19', 'PCIAT-PCIAT_20',
                    'SDS-SDS_Total_Raw', 'SDS-SDS_Total_T', 'PreInt_EduHx-computerinternet_hoursday',
                    'sii']


# Select the specified columns from the train DataFrame
train_selected = train[features_to_keep].copy()

In [ ]:
columns_to_drop = []

for column in features_to_keep:
    # Calculate the ratio of NaN values
    nan_count = train[column].isna().sum()
    total_count = len(train[column])
    ratio = nan_count / total_count
    print(f"The ratio for {column} is {ratio}")
    
    if ratio > 0.7:
        columns_to_drop.append(column)

# Drop the columns with NaN ratio greater than 0.7
train_selected.drop(columns=columns_to_drop, inplace=True)

print(f"Columns dropped: {columns_to_drop}")

## Strategy

We can make category per Age/sex

for float take median 

In [ ]:
# Identify float type columns
float_columns = train_selected.select_dtypes(include=['float64']).columns

print(float_columns)

# Group by 'Basic_Demos-Age' and 'Basic_Demos-Sex'
grouped = train_selected.groupby(['Basic_Demos-Age', 'Basic_Demos-Sex'])

# Function to fill NaN values with the median of the group
def fill_na_with_median(group):
    for column in float_columns:
        median_value = group[column].median()
        group[column].fillna(median_value, inplace=True)
    return group

# Apply the function to each group
train_selected = grouped.apply(fill_na_with_median)


print(train_selected)

In [ ]:
from sklearn.impute import KNNImputer

# Drop ID columns or any non-relevant columns

ids = train_selected['id']
train_selected_without_ids = train_selected.drop(['id'], axis=1)

# Initialize the KNNImputer
imputer = KNNImputer(n_neighbors=20)

# Fit and transform the data
imputed_data = imputer.fit_transform(train_selected_without_ids)

# Optionally, convert the result back to a DataFrame
imputed_df = pd.DataFrame(imputed_data, columns=train_selected_without_ids.columns)

imputed_df['id'] = ids.values

In [ ]:
features_to_keep = imputed_df.columns

for column in features_to_keep:
    # Calculate the ratio of NaN values
    nan_count = imputed_df[column].isna().sum()
    total_count = len(train[column])
    ratio = nan_count / total_count
    print(f"The ratio for {column} is {ratio}")